In [15]:
import requests
import re
import pandas as pd
import jieba
import wordcloud
from bs4 import BeautifulSoup
import time
import json
from stylecloud import gen_stylecloud
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np
import imageio


drama_id = 'https://www.missevan.com/drama/40064' #输入剧集地址
drama_url = 'https://www.missevan.com/dramaapi/getdrama?drama_id= '
url = drama_url+drama_id.split("/")[4]
print(url)
drama_result = requests.get(url).text
#print(drama_result)
patternx = re.compile('"episode":(.+?)"cvs"')
drama_result2= re.findall(patternx,drama_result)
#print(drama_result2)

pattern0 = re.compile('name":(.+?),"sound_id')
sound_name = re.findall(pattern0,str(drama_result2))
#print(sound_name)

pattern1 = re.compile('sound_id":(.+?),"type')
pattern2 = re.compile('"need_pay":(.+?),"')
sound_id = re.findall(pattern1, drama_result)
#print(sound_id)


pay_type = re.findall(pattern2, drama_result)
pay_type = pay_type[1:]
#print(pay_type)

drama = pd.DataFrame({'sound_id': sound_id,'sound_name': sound_name,'pay_type': pay_type})
print(drama)

free_id = list(drama['sound_id'][drama['pay_type']=='0'])
sound_id = list(drama['sound_id'][drama['pay_type']=='1'])
#print(sound_id)
#print(free_id)

sound_url1 ="https://www.missevan.com/sound/"

free_url = 'https://www.missevan.com/sound/getdm?soundid='
for k in range(0,len(free_id)):
    urlfree=free_url+free_id[k]
    urlsound=sound_url1+free_id[k]
    free_result = requests.get(urlfree).text
    free_result1 = requests.get(urlsound).text
    patternf = re.compile(r'<d p="(.+?)</d>')
    dmfree = re.findall(patternf, free_result)
    
    patterntitle = re.compile(r'<title>(.+?)</title>')
    name_result = re.findall(patterntitle, free_result1)

    if k==0: 
        dmfree1=pd.DataFrame({'dm': dmfree})         
        dmsf=dmfree1['dm'].str.split('">',expand=True)[0] 
        dmwf=dmfree1['dm'].str.split('">',expand=True)[1] 
        uidf=dmsf.str.split(',',expand=True)[6] 
        dmfree1['uid']=uidf
        dmfree1['dmw']=dmwf
        print("本集剧集ID为:",free_id[k],"\n本集标题为:",name_result,"\n本集实时ID为:",len(dmfree1['uid'].unique()))

    else:    
        dmfree2=pd.DataFrame({'dm': dmfree}) 
        dmsf_k=dmfree2['dm'].str.split('">',expand=True)[0] 
        dmwf_k=dmfree2['dm'].str.split('">',expand=True)[1] 
        uidf_k=dmsf_k.str.split(',',expand=True)[6] 

        dmfree2['uid']=uidf_k 
        dmfree2['dmw']=dmwf_k
        print("本集剧集ID为:",free_id[k],"\n本集标题为:",name_result,"\n本集实时ID为:",len(dmfree2['uid'].unique()))

        dmfree=dmfree.append(dmfree2) 

sound_url ='https://www.missevan.com/sound/getdm?soundid='
for i in range(0,len(sound_id)): 
    url=sound_url+sound_id[i] 
    urls=sound_url1+sound_id[i]
    sound_result1 = requests.get(urls).text
#    print(sound_result1)
    sound_result = requests.get(url).text 

    patterntitle = re.compile(r'<title>(.+?)</title>')
    name_result = re.findall(patterntitle, sound_result1)
    
    pattern3 = re.compile(r'p="(.+?)"') 
    pattern4 = re.compile(r'<d p="(.+?)</d>')

    dm = re.findall(pattern3, sound_result) 
    dm4 = re.findall(pattern4, sound_result) 
    if i==0: 
        dm1=pd.DataFrame({'dm': dm4})         
        dms=dm1['dm'].str.split('">',expand=True)[0] 
        dmw=dm1['dm'].str.split('">',expand=True)[1] 
        uid=dms.str.split(',',expand=True)[6] 
        dm1['uid']=uid
        dm1['dmw']=dmw
        print("本集剧集ID为:",sound_id[i],"\n本集标题为:",name_result,"\n本集实时ID为:",len(dm1['uid'].unique()))
    else:    
        dm2=pd.DataFrame({'dm': dm4}) 
        dms_i=dm2['dm'].str.split('">',expand=True)[0] 
        dmw_i=dm2['dm'].str.split('">',expand=True)[1] 
        uid_i=dms_i.str.split(',',expand=True)[6] 

        dm2['uid']=uid_i 
        dm2['dmw']=dmw_i
        print("本集剧集ID为:",sound_id[i],"\n本集标题为:",name_result,"\n本集实时ID为:",len(dm2['uid'].unique()))
        dm1=dm1.append(dm2) 

'''
with open('弹幕.txt',mode='a',encoding='utf-8')as f:
    for j in range(0,len(dm1['uid'])):
#    print(dm1['dmw'][j])
        f.write(dm1['dmw'][j])
        f.write('\n')

py=imageio.imread('python.png')
f=open('弹幕.txt',encoding='utf-8')
txt=f.read()
txt_list=jieba.lcut(txt)
string=''.join(txt_list)
#print(string)
wc=wordcloud.WordCloud(width=500,height=500,background_color='white',
                       font_path='msyh.ttc',mask=py,stopwords={'了','这个','我','啊','的'},collocations=False)
wc.generate(string)
wc.to_file('output2.png')
'''                       
                       


print("本剧实时付费ID为:",len(dm1['uid'].unique()))
print("本剧实时付费弹幕数量为:",len(dm1['uid']))


https://www.missevan.com/dramaapi/getdrama?drama_id= 29187
   sound_id        sound_name pay_type
0   3766184             "预告一"        0
1   3779597             "预告二"        0
2   3797447             "第一集"        0
3   3810176         "主题曲《长宴》"        0
4   3826810             "第二集"        0
5   3836026       "主题曲《长宴》PV"        0
6   3848630             "花絮一"        0
7   3856118             "第三集"        1
8   3890896             "第四集"        1
9   3922207             "第五集"        1
10  3953295             "第六集"        1
11  3982466             "第七集"        1
12  4013809             "第八集"        1
13  4045081             "小剧场"        1
14  4045495             "第九集"        1
15  4072484   "100w福利·白噪音·温瀛"        1
16  4072472  "100w福利·白噪音·凌祈宴"        1
17  4077001             "第十集"        1
18  4109048            "第十一集"        1
19  4177487             "花絮二"        1
20  4192140             "小剧场"        1
21  4280266            "完结FT"        1
本集剧集ID为: 3766184 
本集标题为: ['《温香艳玉》第一季 预告一_M站（